In [7]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
AUCs = pd.read_csv('AUCs.txt', sep = '\t')

In [5]:
AUCs.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [6]:
AUCs.describe()

,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
count,14.000000,14.000000,14.000000,14.000000
mean,0.804929,0.820429,0.808786,0.827214
std,0.160187,0.158583,0.167566,0.154548
min,0.583000,0.583000,0.563000,0.569000
25%,0.636250,0.666500,0.624000,0.673000
50%,0.828500,0.863000,0.876000,0.886500
75%,0.950500,0.968750,0.960250,0.965750
max,1.000000,1.000000,1.000000,1.000000


In [11]:
AUCs.columns

Index(['Unnamed: 0', 'C4.5', 'C4.5+m', 'C4.5+cf', 'C4.5+m+cf'], dtype='object')

In [13]:
AUCs_4 = AUCs[AUCs.columns.drop('Unnamed: 0')]

In [16]:
pvalues = []

for i, lhs_column in enumerate(AUCs_4.columns):
    for j, rhs_column in enumerate(AUCs_4.columns):
        if i >= j:
            continue
        
        statistic, p = stats.wilcoxon(AUCs_4[lhs_column], AUCs_4[rhs_column])
        pvalues.append([lhs_column, rhs_column, statistic, p])

In [17]:
pvalues_df = pd.DataFrame.from_records(pvalues)
pvalues_df.columns = ['Modification_A', 'Modification_B', 'Statistic', 'p-value']

In [24]:
pvalues_df

,Modification_A,Modification_B,Statistic,p-value
0,C4.5,C4.5+m,6.5,0.010757
1,C4.5,C4.5+cf,43.0,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015874
3,C4.5+m,C4.5+cf,18.0,0.054329
4,C4.5+m,C4.5+m+cf,22.0,0.327826
5,C4.5+cf,C4.5+m+cf,10.5,0.025314


In [21]:
n = np.argmin(pvalues_df['p-value'].values)
mod_a, mod_b = pvalues_df['Modification_A'][n], pvalues_df['Modification_B'][n]

print(mod_a, ',', mod_b)

C4.5 , C4.5+m


In [23]:
count = len(pvalues_df[pvalues_df['p-value'] < 0.05])
count

3

## Задание 5, Метод Холма

In [26]:
from statsmodels.sandbox.stats.multicomp import multipletests 

In [27]:
reject, p_corrected, a1, a2 = multipletests(pvalues_df['p-value'], 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [32]:
count_rejected = len(reject[reject == True])
count_rejected

0

## Задание 6, Метод Бенджамини-Хохберга

In [33]:
reject, p_corrected, a1, a2 = multipletests(pvalues_df['p-value'], 
                                            alpha = 0.05, 
                                            method = 'fdr_bh')

In [34]:
count_rejected = len(reject[reject == True])
count_rejected

2